In [1]:
import torch
from torch import nn
from d2l import torch as d2l

### Max-Pooling


In [23]:
def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean()
    return Y

In [24]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, (2, 2))

tensor([[4., 5.],
        [7., 8.]])

In [4]:
pool2d(X, (2, 2), 'avg')


tensor([[2., 3.],
        [5., 6.]])

### Padding & Stride

In [5]:
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

In [6]:
pool2d = nn.MaxPool2d(3)
# Pooling has no model parameters, hence it needs no initialization
pool2d(X)

tensor([[[[10.]]]])

The stride and padding can be manually specified to override framework defaults if needed.

In [7]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

Of course, we can specify an arbitrary rectangular pooling window with arbitrary height and width respectively, as the example below shows.

In [8]:
pool2d = nn.MaxPool2d((2, 3), stride=(2,3), padding=(0,1))
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

### Multiple Channels

Below, we will concatenate tensors X and X + 1 on the channel dimension to construct an input with 2 channels.

In [18]:
X = torch.cat((X, X + 1), 1)


In [19]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])

### Exercises

1. Implement average pooling through a convolution.

In [32]:
def avgPoolConv2d(X, pool_size):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    K = torch.full(pool_size, 1/(p_h * p_w))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i: i + p_h, j: j + p_w] * K).sum()
    return Y

In [33]:
X = torch.tensor([[0.0, 1.0, 2.0],
                  [3.0, 4.0, 5.0], 
                  [6.0, 7.0, 8.0]])

avgPoolConv2d(X, (2, 2))

tensor([[2., 3.],
        [5., 6.]])

2. Prove that max-pooling cannot be implemented through a convolution alone.

Can be proven with a simple contradiction example

3. Max-pooling can be accomplished using ReLU operations, i.e., $\mathrm{ReLU}(x) = \max(0, x)$.
- Express $\max (a, b)$ by using only ReLU operations.

In [68]:
def max(a,b):
    return (b if nn.ReLU()(torch.tensor(b-a)) else a)


In [69]:
max(3,1)

3

- Use this to implement max-pooling by means of convolutions and ReLU layers.